In [1]:
%pylab widget
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [3]:
# from analysis_functions import *
import fastcat as fc

In [4]:
from scipy.optimize import minimize

# Looking at the bowtie filter

In [6]:
# Now lets look for one of the bowtie air projections
bowtie_volume = np.load('/media/xcite/DATA/MV_CBCT/projections/kv_air.npy')

plt.figure()
plt.imshow(np.mean(bowtie_volume,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.figure()
plt.plot(np.mean(np.mean(bowtie_volume,0),0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Should I make an actual fastCAT aluminum model to see what CsI setting agrees with this?
# First I will see if I can make a small model that is just a brick of aluminum

In [11]:
plt.figure()
from scipy.signal import gaussian
lengths3 = gaussian(2*512,8*128/1.5)
plt.plot(1.9 - 2*lengths3**(9))
xx = np.arange(1024)
plt.plot(1.9/(1 + np.e**(-0.02*(xx-512-256))))
plt.plot(np.mean(bowtie_volume[0],0)/20000)
plt.plot((-np.log(np.mean(bowtie_volume[0],0)/20000)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
from scipy.signal import gaussian
lengths = gaussian(512,128/1.5)*2

In [13]:
s = fc.calculate_spectrum(100, 14, 3, 100,monitor=None)
s.attenuate(0.4,fc.get_mu(z=13))
s.attenuate(0.089,fc.get_mu(z=22))
s.x,s.y = s.get_points()

In [14]:
plt.figure()
plt.plot(s.x,s.y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
edep = np.load('../../fastcat/data/Detectors/CsI-784-micrometer/EnergyDeposition_450microm.npy')

In [16]:
plt.figure()
plt.semilogx(edep[1],edep[0]/edep[1]*1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
np.sum(s.x)/2

3549.8050000000003

In [18]:
np.argmin(np.abs(np.cumsum(s.y)- np.sum(s.y)/2))

66

In [19]:
s.x[66]

59.42777777777778

## Now we know that the mean energy of the 100 kVp beam should be about 44.6

In [20]:
al = np.loadtxt('../../fastcat/data/mu/14.csv',delimiter=',')

In [21]:
al[0,16]

60.0

In [22]:
mu = (al[1,13]-0.4)

print(mu)

2.9458800000000003


\begin{equation}
y = e^{-\mu x}
\end{equation}

\begin{equation}
-ln(y)/\mu = x
\end{equation}

In [25]:
lengths = -4.*np.log(np.mean(np.mean(bowtie_volume[:,250:450],0),0)/21000)/mu

In [26]:
plt.figure()
# lengths2= concatenate((lengths[:512],lengths[511::-1]))*1
ll = lengths
ll[-30:] = ll[-30]
ll[:30] = ll[30]

# lengths += np.linspace(0.05,0.,1024)
plt.plot(lengths)
# np.save('lengths_2.5cm',lengths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
# cut = 50

# xx = np.arange(len(lengths2))

# cut_lengths = np.interp(np.linspace(cut,len(xx)-cut,len(xx)),xx,lengths2)

# figure();plot(cut_lengths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
# dist = np.linspace(-256*0.0784 - 0.0392,256*0.0784 - 0.0392, 512) # TODO: fix this gore!!
# step = 0.05
# dist2 = np.linspace(-256*step - step/2,256*step - step/2, 512) # TODO: fix this gore!!

# xx = np.arange(len(lengths2))

# cut_lengths = np.interp(dist,dist2,lengths2[::2])

# figure();plot(dist,cut_lengths)
# plot(dist2,lengths2[::2],'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
# Now I just have to find out what the attenuation coefficients are and then I can make an array that has them
# In it to just multiply the final output by.

In [28]:
import fastcat as fc
original_energies_keV = np.array([10,20,30, 40, 50 ,60, 70, 80 ,90 ,100 ,300 ,500 ,700, 900, 1000 ,2000 ,4000 ,6000])

In [29]:
lengths2 = lengths 

bowtie_filter = np.zeros((len(original_energies_keV),lengths2.shape[0]))
bowtie_filter2 = np.zeros((len(original_energies_keV),lengths2.shape[0]))

al_map = fc.get_mu(z=14)

for ii in range(len(original_energies_keV)):
    
    bowtie_filter[ii,:] = np.e**(-lengths2*al_map(original_energies_keV[ii]))
    bowtie_filter2[ii,:] = lengths2*al_map(original_energies_keV[ii])
#     print(al_map(original_energies_keV[ii]))

In [30]:
plt.figure()
plt.plot(bowtie_filter.T);
plt.legend(original_energies_keV)
plt.xlabel('pixel')
plt.ylabel('$I/I_0$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$I/I_0$')

In [31]:
plt.figure()
plt.plot(bowtie_filter2.T[::2,:]);

bowtie_smaller = bowtie_filter.T[::2,:]
bowtie_smaller2 = bowtie_filter2.T[::2,:]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
np.save('bowtie_real',bowtie_smaller)
np.save('bowtie_real_lengths',bowtie_smaller2)

In [46]:
# np.save('bowtie9',bowtie_filter.T)
# np.save('bowtie9_lengths',bowtie_filter2.T)

In [33]:
!mv bowtie* ../../fastcat/data/filters/

In [24]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = 10 #self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind],cmap='bone')
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()

fig, ax = plt.subplots(1, 1)

# X = np.array(projections.T[:,:,5:])
X = bowtie_filter.T

tracker = IndexTracker(ax, X)

fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: not enough values to unpack (expected 3, got 2)

In [267]:
lengths = -4*np.log(np.mean(np.mean(bowtie_volume[:,250:450],0),0)/21000)/mu

In [52]:
plt.figure()
# lengths2= concatenate((lengths[:512],lengths[511::-1]))*1
ll = lengths
ll[-30:] = ll[-30]
ll[:30] = ll[30]

# lengths += np.linspace(0,0.5,1024)
plt.plot(lengths)
# np.save('lengths_2.5cm',lengths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
import fastcat.fastcat as fc
original_energies_keV = np.array([10,20,30, 40, 50 ,60, 70, 80 ,90 ,100 ,300 ,500 ,700, 900, 1000 ,2000 ,4000 ,6000])

In [99]:
lengths2 = lengths

bowtie_filter = np.zeros((len(original_energies_keV),lengths2.shape[0]))
bowtie_filter2 = np.zeros((len(original_energies_keV),lengths2.shape[0]))

al_map = fc.get_mu(z=14)
w_map = fc.get_mu(z=74)

lengths3 = np.zeros(1024)
lengths3[:] = np.linspace(0.002,0.007,1024)

for ii in range(len(original_energies_keV)):
    
    bowtie_filter[ii,:] = np.e**(-lengths2*al_map(original_energies_keV[ii])-lengths3*w_map(original_energies_keV[ii]))
    bowtie_filter2[ii,:] = lengths2*al_map(original_energies_keV[ii])
#     print(al_map(original_energies_keV[ii]))

In [100]:
plt.figure()
plt.plot(bowtie_filter.T);
plt.legend(original_energies_keV)
plt.xlabel('pixel')
plt.ylabel('$I/I_0$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$I/I_0$')

In [101]:
plt.figure()
plt.plot(bowtie_filter.T[::2,:]);

bowtie_smaller = bowtie_filter.T[::2,:]
bowtie_smaller2 = bowtie_filter2.T[::2,:]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
np.save('bowtie_asym2',bowtie_smaller)
np.save('bowtie_asym2_lengths',bowtie_smaller2)

In [103]:
# np.save('bowtie9',bowtie_filter.T)
# np.save('bowtie9_lengths',bowtie_filter2.T)

In [104]:
!mv bowtie* ../../fastcat/data/filters/

In [2]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


# Now lets try to make the flattening filter at a different distance

In [3]:
theta = math.atan(20/150) 

In [7]:
h = math.tan(theta)*15

In [8]:
print(h)

2.0


In [14]:
# lengths = np.load('/home/xcite/fastcat-0.0.1/fastcat/data/filters/bowtie_mod.npy')

In [81]:
figure();plot(lengths[::16])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
lengths_small = lengths[::16]

print(lengths_small)

[2.6776814  2.6902483  2.6788616  2.6787856  2.6722193  2.6638424
 2.6451416  2.6192067  2.5762405  2.5228193  2.4450188  2.3384469
 2.1294308  1.750315   1.4243937  1.1913537  1.0060632  0.8482896
 0.71917766 0.6072884  0.510652   0.425838   0.35118288 0.28545678
 0.23004791 0.18367407 0.14450742 0.10639045 0.0828557  0.06721209
 0.04969179 0.04453822 0.04548175 0.0512171  0.06885744 0.08991212
 0.1218496  0.1537681  0.20050864 0.24711888 0.30504578 0.37096488
 0.44806528 0.5371596  0.64044714 0.7518373  0.88438535 1.0394744
 1.2297472  1.4742594  1.8128681  2.1634922  2.3455245  2.4481015
 2.520307   2.5742002  2.6190462  2.645672   2.6689296  2.6876247
 2.6989727  2.710932   2.722197   2.7372875 ]


In [83]:
print(np.linspace(0,4,64))

[0.         0.06349206 0.12698413 0.19047619 0.25396825 0.31746032
 0.38095238 0.44444444 0.50793651 0.57142857 0.63492063 0.6984127
 0.76190476 0.82539683 0.88888889 0.95238095 1.01587302 1.07936508
 1.14285714 1.20634921 1.26984127 1.33333333 1.3968254  1.46031746
 1.52380952 1.58730159 1.65079365 1.71428571 1.77777778 1.84126984
 1.9047619  1.96825397 2.03174603 2.0952381  2.15873016 2.22222222
 2.28571429 2.34920635 2.41269841 2.47619048 2.53968254 2.6031746
 2.66666667 2.73015873 2.79365079 2.85714286 2.92063492 2.98412698
 3.04761905 3.11111111 3.17460317 3.23809524 3.3015873  3.36507937
 3.42857143 3.49206349 3.55555556 3.61904762 3.68253968 3.74603175
 3.80952381 3.87301587 3.93650794 4.        ]


In [30]:
len(lengths_small)

64

In [33]:
np.linspace(0,8,64)

array([0.        , 0.12698413, 0.25396825, 0.38095238, 0.50793651,
       0.63492063, 0.76190476, 0.88888889, 1.01587302, 1.14285714,
       1.26984127, 1.3968254 , 1.52380952, 1.65079365, 1.77777778,
       1.9047619 , 2.03174603, 2.15873016, 2.28571429, 2.41269841,
       2.53968254, 2.66666667, 2.79365079, 2.92063492, 3.04761905,
       3.17460317, 3.3015873 , 3.42857143, 3.55555556, 3.68253968,
       3.80952381, 3.93650794, 4.06349206, 4.19047619, 4.31746032,
       4.44444444, 4.57142857, 4.6984127 , 4.82539683, 4.95238095,
       5.07936508, 5.20634921, 5.33333333, 5.46031746, 5.58730159,
       5.71428571, 5.84126984, 5.96825397, 6.0952381 , 6.22222222,
       6.34920635, 6.47619048, 6.6031746 , 6.73015873, 6.85714286,
       6.98412698, 7.11111111, 7.23809524, 7.36507937, 7.49206349,
       7.61904762, 7.74603175, 7.87301587, 8.        ])

In [ ]:
0.          0.12698413  0.25396825  0.38095238  0.50793651 0.63492063  0.76190476  0.88888889  1.01587302  1.14285714 1.26984127  1.3968254   1.52380952  1.65079365  1.77777778 1.9047619   2.03174603  2.15873016  2.28571429  2.41269841 2.53968254  2.66666667  2.79365079  2.92063492  3.04761905 3.17460317  3.3015873   3.42857143  3.55555556  3.68253968 3.80952381  3.93650794  4.06349206  4.19047619  4.31746032 4.44444444  4.57142857  4.6984127   4.82539683  4.95238095 5.07936508  5.20634921  5.33333333  5.46031746  5.58730159 5.71428571  5.84126984  5.96825397  6.0952381   6.22222222 6.34920635  6.47619048  6.6031746   6.73015873  6.85714286 6.98412698  7.11111111  7.23809524  7.36507937  7.49206349 7.61904762  7.74603175  7.87301587  8.